In [ ]:
# import pandas_profiling as ppf
# dataset.info()
# report = ppf.ProfileReport(dataset[['Adj Close','Volume']])
# report.to_file('report_AdjCloseVolume.md')

In [3]:
import pandas as pd
import torch
import numpy as np
import datetime
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler ##数据归一化

output_size = 7

In [4]:
def preprocess(data,m,n):
    '''
    data: the dataframe of stock price
    m: 前m天预测
    n: 预测几天
    '''
    minMax = MinMaxScaler()    
    data_transformed = minMax.fit_transform(data)

    adj_close = data["Adj Close"].tolist()
    adj_volume = data_transformed[:,-2:]
    #
    res_X = []
    res_y = []
    
    for i in range(0,len(adj_close)-m-n+1):
        res_X.append(adj_volume[i:i+m])
        res_y.append(adj_close[i+m:i+m+n])
    return res_X,res_y,adj_volume[-m:]

In [5]:
dataset = pd.read_csv('/Users/elvis/ITProjects/GitHub/PythonTask/100dayML/100_day_ml_py/ml_project/raw_price_train/1_r_price_train.csv',index_col='Date')
dataset.index = pd.to_datetime(dataset.index)

In [6]:
X, y, final_pred = preprocess(dataset,14,output_size)
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.7)
print((np.array(X_train)).shape)
# print((np.array(y_train)).shape)
# print((np.array(X)).shape)
# print((np.array(y)).shape)
# print((np.array(y[-1])).shape)
print(y[-1])

(566, 14, 2)
[112.24508700000001, 109.356125, 108.679771, 106.75698899999999, 107.57826999999999, 105.298019, 102.44767]


In [7]:
import torch.utils.data as Data

tensor_x_train = torch.Tensor(X_train)
tensor_y_train = torch.Tensor(y_train)

tensor_x_test = torch.Tensor(X_test)
tensor_y_test = torch.Tensor(y_test)

# 先转换成 torch 能识别的 Dataset
torch_train_dataset = Data.TensorDataset(tensor_x_train, tensor_y_train)
torch_test_dataset = Data.TensorDataset(tensor_x_test, tensor_y_test)

# 把 dataset 放入 DataLoader
train_loader = Data.DataLoader(
    dataset=torch_train_dataset,      # torch TensorDataset format
    batch_size=100,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=2,              # 多线程来读数据
)
test_loader = Data.DataLoader(
    dataset=torch_test_dataset,      # torch TensorDataset format
    batch_size=100,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=2,              # 多线程来读数据
)

In [8]:
import torch.nn as nn
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(  # input shape ( None, 14, 2)
            nn.Conv1d(
                in_channels=14,      # input height
                out_channels=16,    # n_filters
                kernel_size=5,      # filter size
                stride=1,           # filter movement/step
                padding=2,      # 如果想要 con2d 出来的图片长宽没有变化,当 stride=1 padding=(kernel_size-1)/2
            ),
            nn.ReLU(),    # activation
            nn.MaxPool1d(kernel_size=2),    # 在 2x2 空间里向下采样, output shape (batch_size, 16, 1)
        )
        self.out = nn.Sequential(
#             nn.Linear(256*190, 256),
#             nn.Dropout(dropout),
#             nn.ReLU(),
            nn.Linear(16,output_size),
        )

    def forward(self, x):
        conv1ed = self.conv1(x)  # 展平多维的卷积图成 (batch_size, 16, 1)
#         print('conv1ed size ',conv1ed.size())
        conv1ed = conv1ed.view(conv1ed.size(0), -1)
#         print('conv1ed viewed size',conv1ed.size())
        output = self.out(conv1ed)
        return output.squeeze()


In [10]:
model = CNN()
import torch.optim as optim

# 定义优化器
optimizer = optim.Adam(model.parameters())
loss_func = nn.MSELoss()
model

CNN(
  (conv1): Sequential(
    (0): Conv1d(14, 16, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Sequential(
    (0): Linear(in_features=16, out_features=7, bias=True)
  )
)

In [9]:
from sklearn.metrics import mean_squared_error
def train(model, loader, optimizer, loss_func):
    model.train()

    for step, (batch_x, batch_y) in enumerate(loader):  # 每一步 loader 释放一小批数据用来学习
        # 训练的地方...
        batch_x = batch_x.view(-1, 14, 2)              # reshape x to (batch, time_step, input_size)
        batch_y = batch_y.view(-1,output_size)
        output = model(batch_x)
        loss = loss_func(output, batch_y)                   # cross entropy loss
        
        optimizer.zero_grad()                           # clear gradients for this training step
        loss.backward()                                 # backpropagation, compute gradients
        optimizer.step()
        
        acc = mean_squared_error(batch_y.numpy().tolist(),output.detach().numpy())
        
    return loss.data.numpy(), acc

In [10]:
# 不用优化器了
def evaluate(model, loader, loss_func):
    # 转成测试模式，冻结dropout层或其他层
    model.eval()

    with torch.no_grad():
        for step, (batch_x, batch_y) in enumerate(loader):  # 每一步 loader 释放一小批数据用来学习

            batch_x = batch_x.view(-1, 14, 2)              # reshape x to (batch, time_step, input_size)
            batch_y = batch_y.view(-1,output_size)
            output = model(batch_x)
            loss = loss_func(output, batch_y)
            acc = mean_squared_error(batch_y.numpy().tolist(),output.detach().numpy())

        # 调回训练模式
        model.train()

    return acc

In [11]:
best_test_mse = float('inf')
for epoch in range(1501):   # 训练所有!整套!数据 10 次
    train_loss, train_mse = train(model, train_loader, optimizer, loss_func)
    test_mse = evaluate(model, test_loader, loss_func)
    
    
    if test_mse < best_test_mse:
        best_test_mse = test_mse
        torch.save(model.state_dict(), 'stock-cnn-model.pt')
        
    if epoch % 100 == 0 :
        print('Epoch: ', epoch, '| train_loss: ', train_loss, '| train_mse x: ', train_mse, '| test_mse x: ', test_mse)
        # 打出来一些数据
#         print('Epoch: ', epoch, '| Step: ', step, '| batch x: ',
#               batch_x.numpy(), '| batch y: ', batch_y.numpy())

Epoch:  0 | train_loss:  7596.5405 | train_mse x:  7596.54031646347 | test_mse x:  7796.711301318047
Epoch:  100 | train_loss:  604.98804 | train_mse x:  604.9880626870462 | test_mse x:  626.1775160397635
Epoch:  200 | train_loss:  129.85454 | train_mse x:  129.85453121285104 | test_mse x:  129.47517788692963
Epoch:  300 | train_loss:  107.002625 | train_mse x:  107.00262587543439 | test_mse x:  131.98492662412542
Epoch:  400 | train_loss:  142.62404 | train_mse x:  142.62403500714632 | test_mse x:  120.2609931140856
Epoch:  500 | train_loss:  115.90116 | train_mse x:  115.90115841546368 | test_mse x:  97.73263537003827
Epoch:  600 | train_loss:  94.105385 | train_mse x:  94.10538662071824 | test_mse x:  95.90734004068695
Epoch:  700 | train_loss:  95.53449 | train_mse x:  95.53449369819292 | test_mse x:  66.30507067050142
Epoch:  800 | train_loss:  48.00814 | train_mse x:  48.008140478289 | test_mse x:  57.287027766373754
Epoch:  900 | train_loss:  30.900488 | train_mse x:  30.9004887

In [14]:
model.load_state_dict(torch.load('stock-cnn-model.pt'))
pred = model(torch.Tensor(final_pred).view(1,14,2))
print(pred)

tensor([107.7120, 107.7185, 107.7346, 107.7106, 107.5657, 107.6164, 107.5342],
       grad_fn=<SqueezeBackward0>)
